In [10]:
### Python Packages
#This cell imports all the necessary Python libraries for data processing, visualization, 
#natural language processing (NLP), and machine learning. 
#It ensures all dependencies are loaded at the start for seamless notebook execution.

In [11]:
# Used Packages
import pandas as pd         # For data manipulation and handling
import numpy as np          # For numerical computations
import matplotlib.pyplot as plt  # For visualizations
import seaborn as sns       # For enhanced visualizations
from sklearn.model_selection import train_test_split  # For splitting datasets
from sklearn.feature_extraction.text import TfidfVectorizer  # For text vectorization
from sklearn.linear_model import LogisticRegression  # For training the ML model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  # For evaluation
import nltk                 # For natural language processing (NLP)

# Download necessary NLTK data
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pouyasmac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
# Load datasets
fake_df = pd.read_csv('data/fake.csv')  # Replace 'data/' with the path to your fake.csv file
true_df = pd.read_csv('data/true.csv')  # Replace 'data/' with the path to your true.csv file

# Display the first few rows of each dataset
print("Fake News Dataset:")
print(fake_df.head())



Fake News Dataset:
                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  


In [19]:
print("\nTrue News Dataset:")
print(true_df.head())


True News Dataset:
                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  
0  December 31, 2017   
1  December 29, 2017   
2  December 31, 2017   
3  December 30, 2017   
4  December 29, 2017   
